# Training of various models for the project

Once our dataset is in the spectrogram image format (from 'Dataset_Generation.ipynb'), we can train various classifiers (here handmade CNN and transfer learning CNN) using this dataset. 

In this file, we have only the training scripts that we've used to train and evaluate our Deep Learning models to classify between Male and Female speakers from an audio recording of them speaking.

**Note: This is not for running, though it is how we trained our models.**

In [ ]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
import random
import shutil
import glob
import torch
from torchvision import datasets, transforms
from torch.autograd import Variable

## Implementing and running the training model for CNN

In [ ]:
class Net(nn.Module):
    """ 
    Implementing the neural network
    """
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        
net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
# training: loop over the dataset multiple times (whil saving weights)

for epoch in range(20):  
    print("epoch number")
    print(epoch)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        print(i)
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    PATH = '/mntDrive/My Drive/Vox_celeb/VoxCeleb_theo/images/model'
    torch.save(net.state_dict(), str(PATH+str(epoch)+'.pth'))

print('Finished Training')

## Implementing and running the training model for transfer learning

In [ ]:
images , labels = next(iter(train_loader))
images.shape

In [ ]:
for param in model.parameters():
    param.required_grad = False

In [ ]:
num_of_inputs = model.classifier[0].in_features
num_of_inputs

In [ ]:
# restructuring the classifier
import torch.nn as nn
model.classifier = nn.Sequential(
                      nn.Linear(num_of_inputs, 2),
                        nn.LogSoftmax(dim=1))

In [ ]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

In [ ]:
# loss function and optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)


In [ ]:
#loop over the dataset

for epoch in range(n_epochs):
    print(epoch)
    # monitor training loss
    train_loss = 0.0
    train_accuracy = 0
    i=0

    # train the model #
    model.train() # prep model for training
    for data, target in train_loader:
        i+=1
        print(i)
        if train_on_gpu:
            data, target = Variable(data.cuda()), Variable(target.cuda())
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        #calculate accuracy
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
# calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch+1, 
            train_loss
            ))
    print(f"Train accuracy: {train_accuracy/len(train_loader):.3f}")

In [ ]:
train_accuracy/len(train_loader)

In [ ]:
test_accuracy = 0
model.eval() # prep model for evaluation
for data, target in test_loader:
    if train_on_gpu:
        data, target = Variable(data.cuda()), Variable(target.cuda())
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    #calculate accuracy
    ps = torch.exp(output)
    top_p, top_class = ps.topk(1, dim=1)
    equals = top_class == target.view(*top_class.shape)
    test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

print(f"Test accuracy: {test_accuracy/len(test_loader):.3f}")